In [250]:
%%capture
%pip install pandas

In [251]:
import pandas as pd

In [252]:
df = pd.read_csv('raw/combined-original.csv')

In [253]:
def show_info(series: pd.Series):
	print(series.describe())
	print(series.value_counts())
	print(series.isnull().sum())

In [254]:
df.dtypes

Unnamed: 0             int64
Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum               object
ActualElapsedTime    float64
CRSElapsedTime       float64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance             float64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
CancellationCode     float64
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtype: object

In [255]:
print(df.head())

   Unnamed: 0  Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  \
0           0  1987     11          28          6   2042.0        2025   
1           1  1987     10          22          4   1805.0        1800   
2           2  1987     11          19          4    657.0         655   
3           3  1987     11          23          1   1848.0        1835   
4           4  1987     11          25          3    703.0         655   

   ArrTime  CRSArrTime UniqueCarrier  ...  TaxiIn TaxiOut  Cancelled  \
0   2105.0        2053            PI  ...     NaN     NaN          0   
1   1933.0        1905            HP  ...     NaN     NaN          0   
2    821.0         804            PI  ...     NaN     NaN          0   
3   1944.0        1920            DL  ...     NaN     NaN          0   
4    828.0         821            AA  ...     NaN     NaN          0   

   CancellationCode  Diverted  CarrierDelay  WeatherDelay NASDelay  \
0               NaN         0           NaN         

In [256]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [257]:
df.rename(columns={'DayOfWeek': 'DiaSemana', 'UniqueCarrier': 'CompanhiaResponsavel'}, inplace=True)

In [258]:
df.rename(columns={'DayofMonth': 'Day'}, inplace=True)
df['Data'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

In [259]:
# quero ver se Data possui algum NaN

print(df['Data'].isnull().sum())

# não possui

0


In [260]:
# Acabei percebendo que CancellationCode só possui valores NaN

df.drop(['CancellationCode'], axis=1, inplace=True)

In [261]:
# Também reparei que consequentemente nenhum voo foi cancelado

df.drop(['Cancelled'], axis=1, inplace=True)

In [262]:
# Existem colunas vindo como float porém no formato hhmm, bora transformar em hh:mm e manter os NaN como NaN

def format_time(series):
    def process(val):
        if pd.isna(val):
            return None, None
        onlynumber = f'{int(val):04d}'
        return f'{onlynumber[:2]}:{onlynumber[2:]}:00', int(onlynumber[:2]) * 60 + int(onlynumber[2:])
    
    readable, number = zip(*series.apply(process))
    
    return {
        'readable': pd.Series(readable, index=series.index),
        'number': pd.Series(number, index=series.index)
    }

In [263]:
depTime = format_time(df['DepTime'])
df['HoraPartida'] = depTime['readable']
df['HoraPartidaMinutos'] = depTime['number']

crsDepTime = format_time(df['CRSDepTime'])
df['HoraPartidaPrevista'] = crsDepTime['readable']
df['HoraPartidaPrevistaMinutos'] = crsDepTime['number']

arrTime = format_time(df['ArrTime'])
df['HoraChegada'] = arrTime['readable']
df['HoraChegadaMinutos'] = arrTime['number']

crsArrTime = format_time(df['CRSArrTime'])
df['HoraChegadaPrevista'] = crsArrTime['readable']
df['HoraChegadaPrevistaMinutos'] = crsArrTime['number']
df.drop(['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime'], axis=1, inplace=True)

In [264]:
# Vamos checar a relação entre esses valores:

df[['TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',  'HoraPartidaPrevista', 'HoraPartida', 'HoraChegadaPrevista', 'HoraChegada', 'ArrDelay', 'DepDelay']]

# Como podemos:
# - Os valores de atraso podem dizer respeito tanto ao atraso à hora de partida quanto ao de chegada
# - Também existe um delay adicional não calculado
# - Os valores de TaxiIn e TaxiOut já estão inclusos na hora prevista, ou seja, não são atrasos

,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,HoraPartidaPrevista,HoraPartida,HoraChegadaPrevista,HoraChegada,ArrDelay,DepDelay
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:25:00,20:42:00,20:53:00,21:05:00,12.0,17.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:00:00,18:05:00,19:05:00,19:33:00,28.0,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06:55:00,06:57:00,08:04:00,08:21:00,17.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18:35:00,18:48:00,19:20:00,19:44:00,24.0,13.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06:55:00,07:03:00,08:21:00,08:28:00,7.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,5.0,10.0,NaN,NaN,NaN,NaN,NaN,07:30:00,07:31:00,09:05:00,09:06:00,1.0,1.0
21996,2.0,8.0,NaN,NaN,NaN,NaN,NaN,15:30:00,15:41:00,16:30:00,16:36:00,6.0,11.0
21997,6.0,28.0,129.0,0.0,0.0,0.0,0.0,12:00:00,14:22:00,16:50:00,18:59:00,129.0,142.0
21998,2.0,11.0,NaN,NaN,NaN,NaN,NaN,16:40:00,16:44:00,17:45:00,17:46:00,1.0,4.0


In [265]:
df.rename(columns={
    'CarrierDelay': 'AtrasoCompanhia',
	'WeatherDelay': 'AtrasoClimaTempo',
	'NASDelay': 'AtrasoSistemaAereo',
	'SecurityDelay': 'AtrasoSeguranca',
	'LateAircraftDelay': 'AtrasoAeronave',
	'ArrDelay': 'AtrasoChegada',
	'DepDelay': 'AtrasoPartida'
}, inplace=True)
df.fillna({
    'AtrasoCompanhia': 0,
	'AtrasoClimaTempo': 0,
	'AtrasoSistemaAereo': 0,
	'AtrasoSeguranca': 0,
	'AtrasoAeronave': 0,
	'AtrasoChegada': 0,
	'AtrasoPartida': 0
}, inplace=True)

In [266]:
df.rename(columns={
    'Origin': 'AeroportoOrigem',
    'Dest': 'AeroportoDestino',
    'Diverted': 'DesviouRota',
    'AirTime': 'TempoVoando',
    'ActualElapsedTime': 'TempoTotal',
    'CRSElapsedTime': 'TempoTotalPrevisto',
    'Distance': 'Distancia',
    'TailNum': 'NumeroAeronave'
}, inplace=True)
df.drop(['FlightNum'], axis=1, inplace=True)

In [267]:
df.columns

Index(['DiaSemana', 'CompanhiaResponsavel', 'NumeroAeronave', 'TempoTotal',
       'TempoTotalPrevisto', 'TempoVoando', 'AtrasoChegada', 'AtrasoPartida',
       'AeroportoOrigem', 'AeroportoDestino', 'Distancia', 'TaxiIn', 'TaxiOut',
       'DesviouRota', 'AtrasoCompanhia', 'AtrasoClimaTempo',
       'AtrasoSistemaAereo', 'AtrasoSeguranca', 'AtrasoAeronave', 'Data',
       'HoraPartida', 'HoraPartidaMinutos', 'HoraPartidaPrevista',
       'HoraPartidaPrevistaMinutos', 'HoraChegada', 'HoraChegadaMinutos',
       'HoraChegadaPrevista', 'HoraChegadaPrevistaMinutos'],
      dtype='object')

In [268]:
df.dtypes

DiaSemana                              int64
CompanhiaResponsavel                  object
NumeroAeronave                        object
TempoTotal                           float64
TempoTotalPrevisto                   float64
TempoVoando                          float64
AtrasoChegada                        float64
AtrasoPartida                        float64
AeroportoOrigem                       object
AeroportoDestino                      object
Distancia                            float64
TaxiIn                               float64
TaxiOut                              float64
DesviouRota                            int64
AtrasoCompanhia                      float64
AtrasoClimaTempo                     float64
AtrasoSistemaAereo                   float64
AtrasoSeguranca                      float64
AtrasoAeronave                       float64
Data                          datetime64[ns]
HoraPartida                           object
HoraPartidaMinutos                     int64
HoraPartid

In [269]:
df.to_csv('trusted/combined.csv', index=False)